Copyright Matlantis Corp. as contributors to Matlantis contrib project

In [1]:
import os
import glob
import pandas as pd
import re

In [2]:
from ase.io import read, write
from pfcc_extras import view_ngl

## 1. Data extraction from raw data
The raw data was obtained from [QM9NMR dataset](https://moldis-group.github.io/qm9nmr/).

In [3]:
path_list = glob.glob("./raw_data/71Zaisc1SPORF1wnneyUtg/*/*") + glob.glob("./raw_data/gHm0Zu65Q--8x91ap90IMg/*/*") + \
            glob.glob("./raw_data/ry7E8IsyRcGlvOyWZkxfTQ/*/*") + glob.glob("./raw_data/V1FvXN_2ScasSB36-oDzeQ/*/*")
len(path_list)

130831

In [4]:
os.makedirs("molecule_csv", exist_ok=True)

### Extract structure and value from the dataset

In [ ]:
for idx, path in enumerate(path_list):
    if idx % 1000 == 0:
        print(idx)
    
    molecule_id = "m" + path.split("_")[-1][:-4]

    # Extract value
    with open(path) as f:
        cnt = f.read()

    str_list = cnt.split("\n")
    extracted_list = [s for s in str_list if re.match('.*Anisotropy =.*', s)]
    # not Anisotoropy, but isotoropy was used with correction
    info_list = [[line[9], float(line[26:37].strip())] for line in extracted_list]

    df = pd.DataFrame(info_list, columns=["ELEMENT", "VALUE"]).reset_index().rename(columns={"index": "ATOM_IDX"})
    df["MOLECULE_IDX"] = molecule_id
    df.to_csv(f"molecule_csv/{molecule_id}.csv", index=False)
    
    #Extract strucutre
    atoms = read(path)
    atoms.calc = None  # To escape the error, calculator must be None
    write(f"molecule_xyz/{molecule_id}.xyz", atoms)


### Confirm the extracted data

In [6]:
atoms = read("molecule_xyz/m039383.xyz")
print(atoms, len(atoms))
view_ngl(atoms, representations=["ball+stick"])

Atoms(symbols='C3OC5H12', pbc=False) 21
